In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import tqdm
import os

%matplotlib notebook

In [2]:
# Device Configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
# Hyper-parameters 
num_epochs = 10
batch_size = 8
learning_rate = 0.001

In [93]:
class CustomDataset(Dataset):
    def __init__(self, csv_path):
        # Load your data from CSV file
        self.data = pd.read_csv(csv_path)
        self.length = len(self.data)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        # Load and preprocess the data at the given index
        sample = self.data.iloc[index]
        
        # Extract features and labels
        features = torch.tensor(np.reshape(sample.iloc[0:4095].values, (1, 4095)), dtype=torch.float32)  # Adjust based on your column names
#         label = torch.tensor(sample[['y0', 'y1']].values, dtype=torch.float32)  # Assuming label1 and label2 are column names
        label = torch.tensor(np.asarray([sample['y0'], sample['y1']*100]), dtype=torch.float32)

        return features, label

#### Reading Data

In [94]:
dataset = CustomDataset('./data/merged_data.csv')
train_data, val_data = train_test_split(dataset, test_size=0.2)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

In [95]:
for batch_idx, (features, labels) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1} - Features: {features.shape}, Labels: {labels.shape}")
    print("First few elements of features:")
    print(features[:5])
    print("First few elements of labels:")
    print(labels)

    if batch_idx == 0:
        break

Batch 1 - Features: torch.Size([8, 1, 4095]), Labels: torch.Size([8, 2])
First few elements of features:
tensor([[[ 7.0300e-02,  7.5600e-02,  8.0000e-02,  ..., -1.2500e-02,
          -6.9700e-03,  1.6000e-05]],

        [[ 9.8200e-02,  1.0600e-01,  8.0400e-02,  ..., -3.2500e-03,
          -5.4100e-03, -1.9900e-03]],

        [[-9.0300e-03,  7.3200e-03,  1.6700e-03,  ...,  5.5900e-03,
           3.7300e-03, -1.1000e-03]],

        [[ 8.3500e-02,  8.2200e-02,  9.0000e-02,  ...,  2.1700e-03,
           1.0600e-02, -1.6900e-03]],

        [[ 9.2400e-02,  9.4400e-02,  9.7500e-02,  ..., -5.5600e-03,
           2.2400e-03,  9.7900e-04]]])
First few elements of labels:
tensor([[1032.0000,   12.1000],
        [ 982.0000,    4.4000],
        [ 869.0000,    6.2000],
        [ 953.0000,    7.5000],
        [ 953.0000,    5.2000],
        [1010.0000,   10.2000],
        [ 942.0000,    5.5000],
        [1066.0000,    3.9000]])


In [102]:
# # # # To determine the input size of fully connected layer
# dataiter = iter(train_loader)
# images, labels = next(dataiter)

# conv1 = nn.Conv1d(1, 32, 3)
# pool1 = nn.MaxPool1d(5)
# conv2 = nn.Conv1d(32, 64, kernel_size=10)
# pool2 = nn.MaxPool1d(5)
# conv3 = nn.Conv1d(64,128,kernel_size=16)
# print(images.shape)
# y = conv1(images)
# print(y.shape)
# y = pool1(y)
# print(y.shape)
# y = conv2(y)
# print(y.shape)
# y = pool2(y)
# print(y.shape)
# y = conv3(y)
# print(y.shape)

# x = F.relu(conv1(images))
# x = pool1(x)

# x = F.relu(conv2(x))
# x = pool2(x)

# x = F.relu(conv3(x))
# print(x.shape)

#### Model

In [103]:
class RegressionCNN(nn.Module):
    def __init__(self):
        super(RegressionCNN, self).__init__()
        
        self.conv1 = nn.Conv1d(1, 32, 3)
        self.conv2 = nn.Conv1d(32,64,10)
        self.conv3 = nn.Conv1d(64,64,16)
        
        self.pool1 = nn.MaxPool1d(10)
        self.pool2 = nn.MaxPool1d(10)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64*25, 512)  # Adjust input size based on your data
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)  # 2 output values for regression

    def forward(self, x):
        # Convolutional layers with batch normalization and ReLU activation
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv3(x))
        
        x = torch.flatten(x, 1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x


In [104]:
# Instantiate the model
model = RegressionCNN().to(device)

# Print the model architecture
print(model)

RegressionCNN(
  (conv1): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(10,), stride=(1,))
  (conv3): Conv1d(64, 64, kernel_size=(16,), stride=(1,))
  (pool1): MaxPool1d(kernel_size=10, stride=10, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool1d(kernel_size=10, stride=10, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)


In [105]:
model = RegressionCNN().to(device)

In [114]:
# Loss Function
#-----------------------------------------------------
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
# criterion = nn.KLDivLoss(reduction='batchmean')


# Optimizer
#-----------------------------------------------------
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Scheduler
#-----------------------------------------------------
# scheduler = ExponentialLR(optimizer, gamma = 0.1)
# scheduler = StepLR(optimizer, step_size = 4, gamma = 0.5)

In [115]:
# Initialize variable to track loss and accuracies for each epochs
y_loss = {} 
y_loss['train'] = []
y_loss['val'] = []
y_err = {}
y_err['train'] = []
y_err['val'] = []
x_epoch = []

n_total_steps = len(train_loader)
dataset_sizes = {'train': len(train_loader), 'val': len(val_loader)}

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for inputs, targets in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs.to(device))

        # Compute the loss
        loss = criterion(outputs, targets.to(device))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate average training loss
    average_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for val_inputs, val_targets in val_loader:
            val_outputs = model(val_inputs.to(device))
            val_loss += criterion((val_outputs/4096)*, val_targets.to(device)).item()

    # Calculate average validation loss
    average_val_loss = val_loss / len(val_loader)

    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}, Validation Loss: {average_val_loss:.4f}')


with torch.no_grad():
    for val_inputs, val_targets in val_loader:
        val_outputs = model(val_inputs.to(device))
        print(val_outputs)
        print(val_targets.to(device))
        break

Epoch 1/100, Training Loss: 140.8382, Validation Loss: 132.5714
Epoch 2/100, Training Loss: 96.7665, Validation Loss: 114.7989
Epoch 3/100, Training Loss: 95.3681, Validation Loss: 105.0926
Epoch 4/100, Training Loss: 95.4476, Validation Loss: 102.5401
Epoch 5/100, Training Loss: 96.4200, Validation Loss: 115.2832
Epoch 6/100, Training Loss: 86.0831, Validation Loss: 95.3997
Epoch 7/100, Training Loss: 88.5067, Validation Loss: 92.7665
Epoch 8/100, Training Loss: 87.4978, Validation Loss: 112.1006
Epoch 9/100, Training Loss: 96.1083, Validation Loss: 91.9592
Epoch 10/100, Training Loss: 84.2226, Validation Loss: 92.8733
Epoch 11/100, Training Loss: 96.5164, Validation Loss: 99.2793
Epoch 12/100, Training Loss: 103.8447, Validation Loss: 92.8744
Epoch 13/100, Training Loss: 90.9898, Validation Loss: 103.6175
Epoch 14/100, Training Loss: 85.7157, Validation Loss: 91.1573
Epoch 15/100, Training Loss: 81.6151, Validation Loss: 98.7548
Epoch 16/100, Training Loss: 89.1510, Validation Loss: 

In [92]:


# # start
# start = time.time()
# for epoch in tqdm.tqdm(range(num_epochs)):
#     for phase in ['train', 'val']:        
#         running_loss = 0.0
#         running_corrects = 0.0
#         count = 0
#         total = 0
#         if(phase == 'train'):
#             for images, labels in train_loader:
#                 images = images.to(device)
#                 labels = labels.to(device)
        
#                 # Forward pass
#                 outputs = model(images)
#                 _, preds = torch.max(outputs.data, 1)
#                 loss = criterion(outputs, labels)

#                 # Backward and optimize
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()

#                 # the class with the highest energy is what we choose as prediction        
#                 running_loss += loss.item()              
#                 total += labels.size(0)
#                 running_corrects += (preds == labels).sum().item()
        
#             epoch_loss = running_loss / total
#             epoch_acc = running_corrects / total
# #             if (i+1) % 20 == 0:
# #             print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

#             y_loss[phase].append(epoch_loss)
#             y_err[phase].append(1.0 - epoch_acc)
# #             scheduler.step()

            
#         elif(phase == 'val'):
#             for i, (images, labels) in enumerate(val_loader):
#                 images = images.to(device)
#                 labels = labels.to(device)

#                 # Forward pass
#                 outputs = model(images)
#                 _, preds = torch.max(outputs.data, 1)
#                 loss = criterion(outputs, labels)
                
#                 # the class with the highest energy is what we choose as prediction
#                 running_loss += loss.item()              
#                 total += labels.size(0)
#                 running_corrects += (preds == labels).sum().item()               
                
#             epoch_loss = running_loss / total
#             epoch_acc = running_corrects / total
# #             if (i+1) % 200 == 0:
# #             print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

#             y_loss[phase].append(epoch_loss)
#             y_err[phase].append(1.0 - epoch_acc)

#             x_epoch.append(epoch)  
            
# print(f'training time: {(time.time()-start)} sec')
# # end